# Coursera Capstone - Applied Data Science
This nootebok is mainly used for the capstone project.

## week3 : Segmenting and Clustering Neighborhoods in Toronto

I have arranged the three tasks in this assignement in a single file. Ech tasks starts with the caption 'Task' and the respective number of 1 through 3.

### Task 1 : Create a table of postal codes and to transform the data into a pandas dataframe 

In [29]:
# Get the libraries we'll need for this task 

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print('libraries imported')

libraries imported


Load the wikipedia page with Toronto's table of postal codes

In [30]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
tables = pd.read_html(url)

# Let's see how many tables there are on this web page
print(len(tables))

3


Let's have a look at the first table

In [31]:
tables[0].head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [32]:
# Get the first table, because that's the one we want
table = tables[0]

# Extract the columns we want
postal_codes = table[['Postal Code', 'Borough', 'Neighbourhood']]

# Print first 5 columns
postal_codes.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [33]:
# Drop lines that are 'Not assigned'
postal_codes.drop(index=postal_codes[postal_codes['Borough'] == 'Not assigned'].index, inplace=True)

# Reset the index of the dataframe, because some lines have been deleted in the previous step
postal_codes.reset_index(drop=True, inplace=True)

# Print first 5 columns
postal_codes.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Combining rows with more than one neighborhood in one postal code area, separated with a comma

In [34]:
# Find duplicate postal codes
duplicate_rows = postal_codes[postal_codes.duplicated('Postal Code')]

if duplicate_rows.empty:
    print('No duplicate Postal Codes found')

# Seems there are no duplicates -- nothing to be done here

No duplicate Postal Codes found


If a cell has a borough but an unassigned neighbourhood, then the neighbourhood will be set to the same name as the borough

In [8]:
# See how many neighbourhoods are 'Not Assigned' yet
if (postal_codes.loc[postal_codes.Neighbourhood == 'Not assigned', 'Neighbourhood'].count() == 0):
    print('All neighbourhoods were already assigned')
    
# Seems there are no unassigned neighbourhoods -- nothing to be done here

All neighbourhoods were already assigned


Print the number of rows and columns in the dataframe

In [35]:
print('Number of (rows, columns) : ', postal_codes.shape)

Number of (rows, columns) :  (103, 3)


### Task 2 :  Get the latitude and the longitude coordinates of each neighbourhood

Load the csv file that has the geographical coordinates of each postal code

In [36]:
url = 'http://cocl.us/Geospatial_data'
geo_coordinates = pd.read_csv(url)

geo_coordinates.head() # Get a glimpse on the provided data

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Assign coordinates to postal_codes

In [37]:
# Merge the two data frames
geo_merged = pd.merge(postal_codes, geo_coordinates, on='Postal Code')

# Rearrange columns
postals = geo_merged[['Postal Code','Borough','Neighbourhood','Latitude','Longitude']]

postals.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### Task 3 : Explore and cluster the neighborhoods in Toronto

Install and import folium

In [38]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Folium installed and imported!')

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.9.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /Users/gudrun/opt/anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    conda-4.9.2                |   py38h50d1736_0         3.1 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    python_abi-3.8             |           1_cp38           4 KB  conda-forge
    vincent-0.4.4              |             py_1     

Get the location of Toronto's center from Wikipedia : 43.741667, -79.373333

In [53]:
toronto_map = folium.Map(location=[43.741667, -79.373333], zoom_start=12)

toronto_map

Get locations containing 'Toronto'

In [42]:
toronto_loc = postals[postals['Borough'].str.contains("Toronto")]
toronto_loc.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [46]:
# Reset the index of the dataframe
toronto_loc.reset_index(drop=True, inplace=True)
toronto_loc.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [47]:
# Just out of curiosity, let's find out how many entries there are in our new dataframe
toronto_loc.shape

(39, 5)

Now let's superimpose the locations onto the map

In [49]:
# instantiate a feature group for the locations in the dataframe
locations = folium.map.FeatureGroup()

# loop through the 100 crimes and add each to the locations feature group
for lat, lng, in zip(toronto_loc.Latitude, toronto_loc.Longitude):
    locations.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add incidents to map
toronto_map.add_child(locations)

Group the markers into clusters of 'Boroughs' - seems to make sense from looking at the data in toronto_loc

In [52]:
from folium import plugins

# let's start with a clean copy of the map of Toronto
toronto_map = folium.Map(location=[43.741667, -79.373333], zoom_start=12)

# instantiate a mark cluster object for the locations in the dataframe
locations = plugins.MarkerCluster().add_to(toronto_map)

# loop through the dataframe and add each data point to the mark cluster
for lat, lng, label, in zip(toronto_loc.Latitude, toronto_loc.Longitude, toronto_loc.Borough):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(locations)

# display map
toronto_map

That's all